In [115]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [116]:
v_medie = pd.read_csv('./export_5t/export_polito_velocita_medie_feriali_nov2024.csv')
traffico = pd.read_csv('./polito_sensors_vittorio/polito_sensors_vittorio_data.csv', index_col=0)
pioggia = pd.read_csv('./meteo/TORINO GIARDINI REALI_p.csv', index_col=0)

In [117]:
pioggia.drop(columns=['Precipitazione classe'], inplace=True)
pioggia.index = pd.to_datetime(pioggia.index, utc=True)
pioggia.rename({'Precipitazione (mm)': 'precip'}, axis=1, inplace=True)

In [118]:
pioggia.head()

,precip
DATA,
2024-01-01 00:00:00+00:00,0.0
2024-01-02 00:00:00+00:00,0.2
2024-01-03 00:00:00+00:00,0.0
2024-01-04 00:00:00+00:00,0.0
2024-01-05 00:00:00+00:00,23.6


In [119]:
traffico.info()

<class 'pandas.core.frame.DataFrame'>
Index: 44870 entries, 0 to 4373
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   device_id                44870 non-null  int64  
 1   data_location_id         44870 non-null  object 
 2   timestamp_record         44870 non-null  object 
 3   key                      44870 non-null  object 
 4   value                    44870 non-null  float64
 5   accuracy                 44870 non-null  int64  
 6   quality_automatic_check  6121 non-null   object 
 7   quality_manual_check     0 non-null      float64
 8   id_local                 44870 non-null  int64  
 9   id_global                44870 non-null  object 
dtypes: float64(2), int64(3), object(5)
memory usage: 3.8+ MB


In [120]:
traffico.describe()

,device_id,value,accuracy,quality_manual_check,id_local
count,44870.000000,44870.000000,44870.000000,0.0,44870.000000
mean,44048.416046,333.057366,77.339492,NaN,44048.416046
std,35628.695902,238.766144,27.632698,NaN,35628.695902
min,1143.000000,0.000000,0.000000,NaN,1143.000000
25%,1145.000000,98.000000,75.000000,NaN,1145.000000
50%,73223.000000,334.000000,82.000000,NaN,73223.000000
75%,73225.000000,524.000000,98.000000,NaN,73225.000000
max,75176.000000,910.000000,100.000000,NaN,75176.000000


In [121]:
traffico.head()

,device_id,data_location_id,timestamp_record,key,value,accuracy,quality_automatic_check,quality_manual_check,id_local,id_global
0,73224,pasta,2024-03-31 22:00:00+00:00,count_all,61.0,80,NaN,NaN,73224,car-73224
1,1143,pasta,2024-03-31 22:00:00+00:00,count_all,116.0,98,NaN,NaN,1143,car-1143
2,73223,pasta,2024-03-31 22:00:00+00:00,count_all,53.0,80,NaN,NaN,73223,car-73223
3,73225,pasta,2024-03-31 22:00:00+00:00,count_all,116.0,80,NaN,NaN,73225,car-73225
4,1145,pasta,2024-03-31 22:00:00+00:00,count_all,217.0,100,NaN,NaN,1145,car-1145


In [122]:
traffico.drop(columns=['data_location_id', 'quality_automatic_check', 'quality_manual_check', 'id_global', 'id_local'], inplace=True)

In [123]:
traffico.head()

,device_id,timestamp_record,key,value,accuracy
0,73224,2024-03-31 22:00:00+00:00,count_all,61.0,80
1,1143,2024-03-31 22:00:00+00:00,count_all,116.0,98
2,73223,2024-03-31 22:00:00+00:00,count_all,53.0,80
3,73225,2024-03-31 22:00:00+00:00,count_all,116.0,80
4,1145,2024-03-31 22:00:00+00:00,count_all,217.0,100


In [124]:
segment = 73224
group = traffico.groupby('device_id').get_group(segment)
segment_73224 = group.copy()
segment_73224.drop(columns=['device_id', 'key'], inplace=True)
segment_73224['timestamp_record'] = pd.to_datetime(segment_73224['timestamp_record'])
segment_73224.set_index('timestamp_record', inplace=True)
segment_73224.sort_index(inplace=True)
segment_73224.head()

,value,accuracy
timestamp_record,,
2024-03-31 22:00:00+00:00,61.0,80
2024-03-31 23:00:00+00:00,40.0,80
2024-04-01 00:00:00+00:00,35.0,80
2024-04-01 01:00:00+00:00,85.0,80
2024-04-01 02:00:00+00:00,104.0,80


In [125]:
daily_traffic_mean = segment_73224.resample('D').mean()
df = daily_traffic_mean.join(pioggia, how='left')
flag_rain = df['precip'] > 1

In [126]:
# Statistiche medie del traffico
traffic_mean_rain  = df.loc[flag_rain,  'value'].mean()
traffic_mean_clear = df.loc[~flag_rain, 'value'].mean()
traffic_dev_rain  = df.loc[flag_rain,  'value'].std()
traffic_dev_clear = df.loc[~flag_rain, 'value'].std()

In [128]:
print("Traffico medio con pioggia:", traffic_mean_rain)
print("Traffico medio senza pioggia:", traffic_mean_clear)
print("Dev. Std. con pioggia:", traffic_dev_rain)
print("Dev. Std. senza pioggia:", traffic_dev_clear)

Traffico medio con pioggia: 396.01130136986296
Traffico medio senza pioggia: 411.46063631274495
Dev. Std. con pioggia: 65.47208733088834
Dev. Std. senza pioggia: 55.017673030256894


In [129]:
bins   = [0, 0.1, 5, 10, 15, 20, 25, 30, np.inf]
labels = ['0', '0.1-5', '5-10', '10-15', '15-20', '20-25', '25-30', '>30']

df['rain_bin'] = pd.cut(df['precip'].fillna(0),
                        bins=bins,
                        labels=labels,
                        right=False) 

stats = df.groupby('rain_bin', observed=True)['value'].agg(
    mean_value='mean',
    std_value='std',
    n_days='count'
)

print(stats)

          mean_value  std_value  n_days
rain_bin                               
0         410.314109  55.253114     204
0.1-5     418.861111  55.159679      42
5-10      398.323718  87.509490      26
10-15     388.937500  56.968441       6
15-20     383.290625  34.355629       8
20-25     387.011905  33.726498       7
25-30     408.618056  46.285643       6
>30       366.535714  50.895019       7
